## Deploying Docker to Azure Web Apps

Based on Mathew's tutorial located [here](https://github.com/msalvaris/batch_shipyard_notebooks/blob/master/cifar_example/train_on_azure_batch_shipyard.ipynb)

Tutorial currently **not working with CNTK Docker Image** because the Docker image is too big for the Web App (examples & tutorials aren't very big):

```
couldn't open temporary file ...: No space left on device
```

Hence, I use a **Python 3.5 Image and build open-mpi + install CNTK from wheel** instead

In [1]:
import os 
from os import path
import json

In [2]:
# Check that docker is working
!docker run --rm hello-world


Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://cloud.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/engine/userguide/



### Step 1. Create WebApp Code

In [ ]:
%%bash
mkdir script
mkdir script/code

In [4]:
%%writefile script/code/model.py
from flask import Flask
import cntk
import pkg_resources
app = Flask(__name__)
print("Something outside of @app.route() is always loaded")

@app.route("/")
def healthy_me():
    return "healthy"

@app.route('/cntk')
def cntk_ver():
    return "CNTK version: {}".format(pkg_resources.get_distribution("cntk").version)

if __name__ == '__main__':
    # This is just for debugging
    app.run(host='0.0.0.0', port=5005)

Overwriting script/code/model.py


In [5]:
!cat script/code/model.py

from flask import Flask
import cntk
import pkg_resources
app = Flask(__name__)
print("Something outside of @app.route() is always loaded")

@app.route("/")
def healthy_me():
    return "healthy"

@app.route('/cntk')
def cntk_ver():
    return "CNTK version: {}".format(pkg_resources.get_distribution("cntk").version)

if __name__ == '__main__':
    # This is just for debugging
    app.run(host='0.0.0.0', port=5005)

In [6]:
%%writefile script/code/requirements.txt
Flask
gunicorn

Overwriting script/code/requirements.txt


In [7]:
!cat script/code/requirements.txt

Flask
gunicorn

### Step 2. Create Azure Registry to host container

In [ ]:
!az login -o table

In [23]:
selected_subscription = "'.....'"

In [24]:
!az account set --subscription $selected_subscription

In [25]:
docker_registry = "ikmscontainer"
docker_registry_group = "ikmscontainergorup"

In [ ]:
!az group create -n $docker_registry_group -l southcentralus -o table

In [ ]:
!az acr create -n $docker_registry -g $docker_registry_group -l southcentralus -o table

In [ ]:
!az acr update -n $docker_registry --admin-enabled true -o table

In [29]:
json_data = !az acr credential show -n $docker_registry
docker_username = json.loads(''.join(json_data))['username']
docker_password = json.loads(''.join(json_data))['password']

In [ ]:
print(docker_username)
print(docker_password)

In [31]:
json_data = !az acr show -n $docker_registry
docker_registry_server = json.loads(''.join(json_data))['loginServer']

### Step 3. Create Docker Image

In [36]:
!mkdir script/docker

In [32]:
# Using CNTK docker doesn't work - too big for WebApp
"""
%%writefile script/docker/dockerfile

FROM microsoft/cntk:2.0.beta15.0-cpu-python3.5
MAINTAINER Ilia Karmanov
ADD code /code
ENV PATH /root/anaconda3/envs/cntk-py35/bin:$PATH
WORKDIR /code
RUN pip install -r requirements.txt && \
    sudo rm -R /cntk/Examples && \
    sudo rm -R /cntk/Tutorials 

EXPOSE 5005
CMD ["python", "model.py"]
"""

'\n%%writefile script/docker/dockerfile\n\nFROM microsoft/cntk:2.0.beta15.0-cpu-python3.5\nMAINTAINER Ilia Karmanov\nADD code /code\nENV PATH /root/anaconda3/envs/cntk-py35/bin:$PATH\nWORKDIR /code\nRUN pip install -r requirements.txt &&     sudo rm -R /cntk/Examples &&     sudo rm -R /cntk/Tutorials \n\nEXPOSE 5005\nCMD ["python", "model.py"]\n'

I don't actually use gunicorn (use flask dev server) but will change later (should be trivial).

Had to use standard python image and build+install open-mpi and then install CNTK from wheel - this generates an image of size 1.63 GB otherwise it's 5.1GB (CNTK Docker Image)

In [53]:
%%writefile script/docker/dockerfile

FROM python:3.5
MAINTAINER Ilia Karmanov
ADD code /code
ENV PATH /usr/local/mpi/bin:$PATH
ENV LD_LIBRARY_PATH /usr/local/mpi/lib:$LD_LIBRARY_PATH
WORKDIR /code
RUN wget https://www.open-mpi.org/software/ompi/v1.10/downloads/openmpi-1.10.3.tar.gz && \
    tar -xzvf ./openmpi-1.10.3.tar.gz && \
    cd openmpi-1.10.3 && \
    ./configure --prefix=/usr/local/mpi && \
    make -j all && \
    make install && \
    cd .. && \
    rm -R openmpi-1.10.3 && \
    rm openmpi-1.10.3.tar.gz && \
    pip install https://cntk.ai/PythonWheel/CPU-Only/cntk-2.0.beta15.0-cp35-cp35m-linux_x86_64.whl && \
    pip install -r requirements.txt

EXPOSE 5005
CMD ["python", "model.py"]

Overwriting script/docker/dockerfile


In [54]:
container_name = docker_registry_server + "/ilkarman/smallapp"
application_path = 'script'
docker_file_location = path.join(application_path, 'docker/dockerfile')

In [55]:
!docker login $docker_registry_server -u $docker_username -p $docker_password

Login Succeeded


In [51]:
# Debug: remove all containers
#%%bash
#docker stop $(docker ps -a -q)
#docker rm $(docker ps -a -q)

In [38]:
# This will take a while; potentially run from shell instead
build_out = !docker build -t $container_name -f $docker_file_location $application_path --no-cache

Test everything is working locally before pushing

In [65]:
#!docker images   

In [58]:
# To debug
print(container_name)
# In shell (run interactive mode):
#docker run -it $container_name /bin/bash
#conda info --env
#which python
# ... etc

ikmscontainer.azurecr.io/ilkarman/smallapp


In [59]:
test_cont = !docker run -p 5005:5005 -d $container_name

In [60]:
import time; time.sleep(5)  # Wait to load
!curl http://0.0.0.0:5005

healthy

In [61]:
!curl http://0.0.0.0:5005/cntk

CNTK version: 2.0.beta15.0

In [62]:
!docker kill {test_cont[0]}

4d672a6e5ba38d352797c1d5f0161b763f8a09aed939aa1fb3ac49ecbbe28a61


### Step 4. Push Docker Image to Registry

In [63]:
!docker push $container_name

The push refers to a repository [ikmscontainer.azurecr.io/ilkarman/smallapp]

5ca7bebd: Preparing 
f1ada4ce: Preparing 
7ec37d7a: Preparing 
835d74bb: Preparing 
16b85b72: Preparing 
de0e3e2b: Preparing 
2eb04a92: Preparing 
80599f68: Preparing 
5ca7bebd: Pushed    944 MB/940.4 MBlatest: digest: sha256:91f31d6a763a323b53fbf473eb19bb24d815b32e7538ec08d48ee5e339780b47 size: 2216


Note: the size of the image, since the Web App loads the container into RAM you would need to configure it to have more RAM than the size of the image (if this is not possible then try to remove unnecessary files in your image or use another).

Note: I'm not sure how much bigger deployed container is compared to compressed image?

### Step 5. Create Azure Web App from Docker Image

Note: Currently cannot find a CLI-way of doing this

1. Go to your Azure Portal and create a new 'Web App on Linux (preview)' resource

2. Click on 'Configure container' and select 'Private registry' under 'Image source'

3. Configure the 'App Service plan/Location' so that your Web App size has enough RAM to host the container (**I find that S2 is enough for the above 1.6GB image, however if you plan to also include a trained-model it would be best to go to S3, otherwise your WebApp will not deploy**)

4. Enter the details to connect to your ACR:
```
Image and optional tag: ikmscontainer.azurecr.io/ilkarman/smallapp
Server URL: http://ikmscontainer.azurecr.io
Login username:<docker_username>
Password:<docker_password>
```

6. Create your Web App

7. Go to the 'Application Settings' blade, scroll down until you see 'App settings' and add an entry (to use whichever port you setup), and click save:
```
Key:PORT
Value:5005
```

8. You should now be able to navigate to your Azure Web App address and see your project! If not - add '.scm' just before '.azurewebsites.net' in your URL e.g. https://cntkiksmall.scm.azurewebsites.net/ to access the Kudu console and go to 'Debug console' -> 'Bash', where you can access logfiles such as:
```
cd /home/LogFiles/docker
ls
```

9. The first load may take a while - this is because the docker image is downloaded to the WebApp. You can observe this by opening the first log-file in your directory:
```
cd /home/LogFiles/docker
cat docker_13_out.log
```
    The output should look like:
    ```
    7d27bd3d7fec: Verifying Checksum
    7d27bd3d7fec: Download complete
    7d27bd3d7fec: Pull complete
    44ae682c18a3: Pull complete
    824bd01a76a3: Pull complete
    68fe59875298: Pull complete
    9ca1d7ae0c4b: Pull complete
    46beba4b643f: Pull complete
    651cd581382c: Pull complete
    Digest: sha256:1efdaef9d8c208753fe36ccff197f28c719cc5f7d0bf5ff12f839f04e76c5f98
    Status: Downloaded newer image for ikmscontainer.azurecr.io/ilkarman/smallapp:latest
    ```




In [67]:
# Success
!curl http://cntkiksmall.azurewebsites.net/

healthy

In [68]:
!curl http://cntkiksmall.azurewebsites.net/cntk

CNTK version: 2.0.beta15.0